In [77]:
from ib_insync import *
from datetime import datetime
from math import log, sqrt, exp
from scipy.stats import norm
import numpy as np

In [78]:
import nest_asyncio
nest_asyncio.apply()

from ib_insync import *

# Connect to IB Gateway or TWS
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)  # use 4002 for paper, 7496 for live

print("Connected:", ib.isConnected())

Connected: True


Error 321, reqId 3: Error validating request.-'co' : cause - Invalid contract id
Error 321, reqId 4: Error validating request.-'co' : cause - Invalid contract id
Error 321, reqId 5: Error validating request.-'co' : cause - Invalid contract id
Error 321, reqId 6: Error validating request.-'co' : cause - Invalid contract id
Error 321, reqId 7: Error validating request.-'co' : cause - Invalid contract id
Error 321, reqId 8: Error validating request.-'co' : cause - Invalid contract id
Error 321, reqId 9: Error validating request.-'co' : cause - Invalid contract id
Error 321, reqId 10: Error validating request.-'co' : cause - Invalid contract id
Error 321, reqId 11: Error validating request.-'co' : cause - Invalid contract id
Peer closed connection.


In [82]:
def get_chain(symbol):
    """Fetch option chain from IBKR for given symbol."""
    stock = Stock(symbol, 'SMART', 'USD')
    chains = ib.reqSecDefOptParams(stock.symbol, '', stock.secType, stock.conId)
    if not chains:
        print(f"No option chain for {symbol}")
        return None
    chain = chains[0]  # use first exchange returned (e.g., CBOE)
    return chain

def get_target_expiries(symbol, target_days=(30, 60)):
    """Return expiries closest to 30 and 60 days from today."""
    chain = get_chain(symbol)
    if not chain or not chain.expirations:
        return []
    
    today = datetime.now().date()
    expiries = [datetime.strptime(d, '%Y%m%d').date() for d in chain.expirations]
    
    results = []
    for td in target_days:
        exp = min(expiries, key=lambda d: abs((d - today).days - td))
        results.append(exp)
    
    return results

def get_atm_iv(symbol, expiry_date):
    """Fetch ATM call IV for the nearest strike."""
    stock = Stock(symbol, 'SMART', 'USD')
    chain = ib.reqSecDefOptParams(stock.symbol, '', stock.secType, stock.conId)
    if not chain:
        return None
    
    chain = chain[0]
    strikes = sorted([s for s in chain.strikes])
    if not strikes:
        return None
    
    # approximate ATM strike
    ib.qualifyContracts(stock)
    stock_price = ib.reqMktData(stock)
    ib.sleep(1)
    S = stock_price.last
    atm_strike = min(strikes, key=lambda k: abs(k - S))
    
    option = Option(symbol, expiry_date.strftime('%Y%m%d'), atm_strike, 'C', 'SMART')
    ib.qualifyContracts(option)
    
    ticker = ib.reqMktData(option, '', False, False)
    ib.sleep(2)  # wait for data
    iv = getattr(ticker, 'impliedVol', None)
    return iv

def forward_vol(iv1, iv2, t1, t2):
    """Compute annualized forward volatility."""
    if iv1 is None or iv2 is None:
        return None
    try:
        fwd_var = ((iv2**2 * t2) - (iv1**2 * t1)) / (t2 - t1)
        if fwd_var < 0:
            return None
        return math.sqrt(fwd_var)
    except:
        return None

# -----------------------
# Main calculation
# -----------------------
def calc_forward_vol(symbol):
    expiries = get_target_expiries(symbol, target_days=(30, 60))
    if not expiries or len(expiries) < 2:
        print(f"No suitable expiries for {symbol}")
        return None
    
    today = datetime.now().date()
    exp30, exp60 = expiries
    
    iv30 = get_atm_iv(symbol, exp30)
    iv60 = get_atm_iv(symbol, exp60)
    
    fwd = None
    if iv30 is not None and iv60 is not None:
        t1 = (exp30 - today).days / 365
        t2 = (exp60 - today).days / 365
        fwd = forward_vol(iv30, iv60, t1, t2)
    
    print(f"{symbol} results:")
    print(f"  {exp30} IV: {iv30}")
    print(f"  {exp60} IV: {iv60}")
    print(f"  Forward Vol: {fwd}")
    print()
    
    return {
        'symbol': symbol,
        'expiry_30D': exp30,
        'iv_30D': iv30,
        'expiry_60D': exp60,
        'iv_60D': iv60,
        'forward_vol': fwd
    }


In [83]:
tickers = ['AAPL', 'MSFT', 'GOOG']
results = []

for t in tickers:
    res = calc_forward_vol(t)
    if res:
        results.append(res)

No option chain for AAPL
No suitable expiries for AAPL
No option chain for MSFT
No suitable expiries for MSFT
No option chain for GOOG
No suitable expiries for GOOG
